In [4]:
import numpy as np

In [90]:
#test array/graph from https://people.sc.fsu.edu/~jburkardt/m_src/rcm/rcm.html
graph = np.array([
            [0, 0, 1, 0, 0, 0, 0, 1, 1],
            [0, 0, 1, 0, 0, 1, 1, 1, 0],
            [1, 1, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 1, 1],
            [0, 0, 0, 0, 0, 0, 1, 1, 0],
            [0, 1, 0, 0, 0, 0, 1, 0, 0],
            [0, 1, 0, 0, 1, 1, 0, 0, 0],
            [1, 1, 0, 1, 1, 0, 0, 0, 0],
            [1, 0, 0, 1, 0, 0, 0, 0, 0]
             ])
print(graph)

[[0 0 1 0 0 0 0 1 1]
 [0 0 1 0 0 1 1 1 0]
 [1 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 1]
 [0 0 0 0 0 0 1 1 0]
 [0 1 0 0 0 0 1 0 0]
 [0 1 0 0 1 1 0 0 0]
 [1 1 0 1 1 0 0 0 0]
 [1 0 0 1 0 0 0 0 0]]


In [43]:
#to generate rand arr
N = 10
b = np.random.randint(2,size=(N,N))
#b_symm = np.array(np.ceil((b + b.T)/2), dtype=int)
#np.fill_diagonal(b_symm, 0)
#b_symm
print(b+b.T)

[[2 0 1 2 0 1 1 0 2 2]
 [0 2 2 1 1 2 0 0 2 0]
 [1 2 2 0 1 2 2 2 1 1]
 [2 1 0 2 0 0 2 1 2 1]
 [0 1 1 0 0 0 0 2 1 1]
 [1 2 2 0 0 0 1 0 2 1]
 [1 0 2 2 0 1 0 2 1 1]
 [0 0 2 1 2 0 2 0 1 0]
 [2 2 1 2 1 2 1 1 0 1]
 [2 0 1 1 1 1 1 0 1 2]]


In [91]:
from scipy.sparse import csr_matrix, csc_matrix
from scipy.sparse.csgraph import reverse_cuthill_mckee

In [92]:
c_g = csr_matrix(graph)
print(c_g)
perm = reverse_cuthill_mckee(c_g, True)
perm

  (0, 2)	1
  (0, 7)	1
  (0, 8)	1
  (1, 2)	1
  (1, 5)	1
  (1, 6)	1
  (1, 7)	1
  (2, 0)	1
  (2, 1)	1
  (3, 7)	1
  (3, 8)	1
  (4, 6)	1
  (4, 7)	1
  (5, 1)	1
  (5, 6)	1
  (6, 1)	1
  (6, 4)	1
  (6, 5)	1
  (7, 0)	1
  (7, 1)	1
  (7, 3)	1
  (7, 4)	1
  (8, 0)	1
  (8, 3)	1


array([4, 3, 6, 5, 7, 8, 1, 0, 2], dtype=int32)

In [93]:
g = graph[perm]
print(g)
print(c_g[perm])

[[0 0 0 0 0 0 1 1 0]
 [0 0 0 0 0 0 0 1 1]
 [0 1 0 0 1 1 0 0 0]
 [0 1 0 0 0 0 1 0 0]
 [1 1 0 1 1 0 0 0 0]
 [1 0 0 1 0 0 0 0 0]
 [0 0 1 0 0 1 1 1 0]
 [0 0 1 0 0 0 0 1 1]
 [1 1 0 0 0 0 0 0 0]]
  (0, 7)	1
  (0, 6)	1
  (1, 8)	1
  (1, 7)	1
  (2, 5)	1
  (2, 4)	1
  (2, 1)	1
  (3, 6)	1
  (3, 1)	1
  (4, 4)	1
  (4, 3)	1
  (4, 1)	1
  (4, 0)	1
  (5, 3)	1
  (5, 0)	1
  (6, 7)	1
  (6, 6)	1
  (6, 5)	1
  (6, 2)	1
  (7, 8)	1
  (7, 7)	1
  (7, 2)	1
  (8, 1)	1
  (8, 0)	1


In [94]:
graph = [
[0, 1 , 2, 0],
[0, 0, 0, 1],
[2, 0, 0, 3],
[0, 0, 0, 0]
]
graph = csr_matrix(graph)
print(graph)


  (0, 1)	1
  (0, 2)	2
  (1, 3)	1
  (2, 0)	2
  (2, 3)	3


In [95]:
perm = reverse_cuthill_mckee(graph)
print(perm)
graph[perm].toarray()

[3 2 1 0]


array([[0, 0, 0, 0],
       [2, 0, 0, 3],
       [0, 0, 0, 1],
       [0, 1, 2, 0]], dtype=int32)